In [1]:
import sys

# setting path
sys.path.append('../')

import livef1
import pandas as pd

In [2]:
# season = livef1.get_season(2021)

meeting = livef1.get_meeting(season=2024, meeting_identifier="Spa")

2024-12-04 16:34:30,796 : livef1 : DEBUG :: Trying to get season 2024.
2024-12-04 16:34:32,804 : livef1 : DEBUG :: The season was received successfully.
2024-12-04 16:34:32,806 : livef1 : DEBUG :: Trying to get meeting.
2024-12-04 16:34:32,815 : livef1 : DEBUG :: Searching of identifier 'Spa' has started.
2024-12-04 16:34:32,824 : livef1 : INFO :: Found at column 'MEETING CIRCUIT SHORTNAME' as 'Spa-Francorchamps'.
2024-12-04 16:34:32,827 : livef1 : INFO :: Selected meeting/session is:
Meeting Offname : FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
Meeting Name : Belgian Grand Prix
Meeting Circuit Shortname : Spa-Francorchamps
2024-12-04 16:34:32,833 : livef1 : DEBUG :: The meeting was received successfully.


meeting_offname              FORMULA 1 ROLEX BELGIAN GRAND PRIX 2024
meeting_name                                      Belgian Grand Prix
meeting_circuit_shortname                          Spa-Francorchamps
Name: 20, dtype: object


In [3]:
type(meeting)

livef1.models.meeting.Meeting

In [8]:
print(meeting.sessions_table.to_markdown())

|    |   Season Year | Meeting Location   | Session Type   | Meeting Code   |   Meeting Key |   Meeting Number | Meeting Offname                         | Meeting Name       |   Meeting Country Key | Meeting Country Code   | Meeting Country Name   |   Meeting Circuit Key | Meeting Circuit Shortname   |   Session Key | Session Name   | Session Startdate   | Session Enddate     | Gmtoffset   | Path                                                      |
|---:|--------------:|:-------------------|:---------------|:---------------|--------------:|-----------------:|:----------------------------------------|:-------------------|----------------------:|:-----------------------|:-----------------------|----------------------:|:----------------------------|--------------:|:---------------|:--------------------|:--------------------|:------------|:----------------------------------------------------------|
|  0 |          2024 | Spa-Francorchamps  | Practice 1     | BEL02012       |          124

In [7]:
help(meeting)

Help on Meeting in module livef1.models.meeting object:

class Meeting(builtins.object)
 |  Meeting(season: 'Season' = None, year: int = None, code: int = None, key: str = None, number: int = None, location: str = None, officialname: str = None, name: str = None, country: Dict = None, circuit: Dict = None, sessions: List = None, loaded: bool = False, **kwargs)
 |  
 |  Represents a meeting in a specific season with relevant details and associated sessions.
 |  
 |  Attributes
 |  ----------
 |  season : :class:`~Season`
 |      The season this meeting belongs to.
 |  year : :class:`int`
 |      The year of the meeting.
 |  code : :class:`int`
 |      The unique code for the meeting.
 |  key : :class:`str`
 |      The unique identifier for the meeting.
 |  number : :class:`int`
 |      The sequential number of the meeting in the season.
 |  location : :class:`str`
 |      The location (e.g., circuit name) of the meeting.
 |  officialname : :class:`str`
 |      The official name of the m

In [12]:
meeting.officialname

'FORMULA 1 ARAMCO GRAN PREMIO DE ESPAÑA 2024'

In [ ]:
help(meeting.sessions_table.__repr__)

Help on method __repr__ in module pandas.core.frame:

__repr__() -> 'str' method of pandas.core.frame.DataFrame instance
    Return a string representation for a particular DataFrame.



: 